In [1]:
import sys
import os
# gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
# sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data_local_computer

import torch
from collections import defaultdict

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy                    # clone tensor


In [2]:
lat_lon_resolution = [15,15]
day = 1
mm_cond_number = 20

years = ['2024']
month_range =[7,8]
idx_for_datamap= [ 8*(day-1),8*day]

instance = load_data_local_computer()
map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap=[0,8])


### quick load from amarel

In [3]:
input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates"

# input_filename = "vecc_extra_estimates_50_july24.pkl"
# input_filename = "vecc_inter_estimates_1250_july24.pkl"

input_filename = "vecc_inter_estimates_5000_july24.pkl"
# input_filename = "estimation_200_july24.pkl"
# input_filename = "estimation_1250_july24.pkl"
input_filepath = os.path.join(input_path, input_filename)
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    amarel_map1250= pickle.load(pickle_file)

# Assuming df_1250 is your DataFrame
df_1250 = pd.DataFrame()
for key in amarel_map1250:
    tmp = pd.DataFrame(amarel_map1250[key][0].reshape(1, -1), columns=['sigmasq', 'range_lat', 'range_lon', 'advec_lat', 'advec_lon', 'beta', 'nugget'])
    tmp['loss'] = amarel_map1250[key][1]
    df_1250 = pd.concat((df_1250, tmp), axis=0)

# Generate date range
date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(df_1250):
    df_1250.index = date_range
else:
    print("The number of dates does not match the number of rows in the DataFrame.")

print(df_1250)
df = df_1250
# Save DataFrame to CSV
output_filename = 'vecchia_inter_estimates_1250_july24.csv'
output_csv_path = os.path.join(input_path, output_filename)
df.to_csv(output_csv_path, index=False)

The number of dates does not match the number of rows in the DataFrame.
     sigmasq  range_lat  range_lon  advec_lat  advec_lon      beta    nugget  \
0  25.822897   1.023014   1.131423   0.073286  -0.095810  0.177670  1.569700   
0  24.242847   2.147953   1.945775   0.050045  -0.070032  0.140178  1.442902   
0  26.156120   0.818184   1.061921   0.081691  -0.119475  0.241490  2.123204   
0  26.001230   1.194200   1.434477  -0.206528  -0.174575  0.041952  2.593778   
0  23.724661   1.819339   2.713623  -0.166419  -0.037361 -0.014584  1.945128   

           loss  
0  50396.417969  
0  43290.984375  
0  54808.964844  
0  52697.542969  
0  43821.054688  


Investigate conditioning number

10 seems best no more no less

In [10]:
sd = analysis_data_map['2024_07_y24m07day01_hm01:00']
# Compute the required statistics
# Compute the required statistics
max_lat = torch.max(sd[:, 0])
min_lat = torch.min(sd[:, 0])
median_lat = torch.median(sd[:, 0])

max_lon = torch.max(sd[:, 1])
min_lon = torch.min(sd[:, 1])
median_lon = torch.median(sd[:, 1])

# Extract the 9 points along with their locations (indices)
points = [
    (min_lon, min_lat),
    #(min_lon, median_lat),
    (min_lon, max_lat),
    (median_lon, min_lat),
    #(median_lon, median_lat),
    (median_lon, max_lat),
    (max_lon, min_lat),
    #(max_lon, median_lat),
    (max_lon, max_lat)
]
print(points)

indices = []
for lon, lat in points:
    condition = (sd[:, 0] == lat) & (sd[:, 1] == lon)
    indices.append(torch.where(condition)[0])

# Create the indices tensor
indices_tensor = torch.cat(indices)

print("Indices in Tensor Frame:")
print(indices_tensor)

base_list = indices_tensor.clone().detach().tolist()

[(tensor(110.0250, dtype=torch.float64), tensor(5.0250, dtype=torch.float64)), (tensor(110.0250, dtype=torch.float64), tensor(9.0250, dtype=torch.float64)), (tensor(114.0250, dtype=torch.float64), tensor(5.0250, dtype=torch.float64)), (tensor(114.0250, dtype=torch.float64), tensor(9.0250, dtype=torch.float64)), (tensor(119.0250, dtype=torch.float64), tensor(5.0250, dtype=torch.float64)), (tensor(119.0250, dtype=torch.float64), tensor(9.0250, dtype=torch.float64))]
Indices in Tensor Frame:
tensor([ 2,  4, 34, 45, 40,  1])


Optimization vecchia

In [ ]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number
)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.99), eps=1e-8, step_size=10, gamma=0.9)  
instance.run_vecc_interpolate(params, optimizer, scheduler,epochs=3000)

Vecchia experiments here

In [ ]:
idx_for_datamap = [0,8]

class matern_advec_beta_torch_vecchia:
    def __init__(self, analaysis_data_map: torch.Tensor, params: torch.Tensor, nns_map=nns_map, mm_cond_number=mm_cond_number):
        
        self.key_list = sorted(analysis_data_map)
        self.input_map = analysis_data_map

        self.mm_cond_number = mm_cond_number
        self.nns_map = nns_map 
        self.input_map = analaysis_data_map
        self.smooth = 0.5  
        sample_df = analaysis_data_map[self.key_list[0]]

        self.size_per_hour = len(sample_df)

    def custom_distance_matrix(self, U, V):
        # Efficient distance computation with broadcasting
        spatial_diff = torch.norm(U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0), dim=2)

        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        distance = (spatial_diff**2 + temporal_diff**2)  # move torch.sqrt to covariance function to track gradients of beta and avec
        return distance
    
    def precompute_coords_ani(self, params, y: torch.Tensor, x: torch.Tensor)-> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params

        if y is None or x is None:
            raise ValueError("Both y and x_df must be provided.")

        x1 = x[:, 0]
        y1 = x[:, 1]
        t1 = x[:, 3]

        x2 = y[:, 0]
        y2 = y[:, 1]
        t2 = y[:, 3]

        # spat_coord1 = torch.stack((self.x1 , self.y1 - advec * self.t1), dim=-1)
        spat_coord1 = torch.stack(( (x1 - advec_lat * t1)/range_lat, (y1 - advec_lon * t1)/range_lon ), dim=-1)
        spat_coord2 = torch.stack(( (x2 - advec_lat * t2)/range_lat, (y2 - advec_lon * t2)/range_lon ), dim=-1)

        U = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
        V = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

        distance = self.custom_distance_matrix(U,V)
        non_zero_indices = distance != 0
        return distance, non_zero_indices
    
    # anisotropic in three 
    def matern_cov_ani(self,params: torch.Tensor, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
        

        distance, non_zero_indices = self.precompute_coords_ani(params, x,y)
        out = torch.zeros_like(distance)

        non_zero_indices = distance != 0
        if torch.any(non_zero_indices):
            out[non_zero_indices] = sigmasq * torch.exp(- torch.sqrt(distance[non_zero_indices]))
        out[~non_zero_indices] = sigmasq

        # Add a small jitter term to the diagonal for numerical stability
        out += torch.eye(out.shape[0], dtype=torch.float64) * nugget 

        return out
    
    def full_likelihood(self,params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, covariance_function) -> torch.Tensor:
        input_arr = input_np[:, :4]
        y_arr = y

        # Compute the covariance matrix
        cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
        
        # Compute the log determinant of the covariance matrix
        sign, log_det = torch.slogdet(cov_matrix)
        #log_det = torch.log(torch.linalg.det(cov_matrix))
        #if sign <= 0:
        #    raise ValueError("Covariance matrix is not positive definite")
        
        # Extract locations
        locs = input_arr[:, :2]

        # Compute beta
        tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
        beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        mu = torch.matmul(locs, beta)
        y_mu = y_arr - mu

        # Compute the quadratic form
        quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))

        # Compute the negative log likelihood
        neg_log_lik = 0.5 * (log_det + quad_form)
        # neg_log_lik = 0.5 * ( log_det )
        return  neg_log_lik 


    def vecchia_local_full_cond(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        neg_log_lik = 0.0

                # Use below when working on local computer to avoid singular matrix
        cur_heads = aggregated_data[:20, :]
        neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

        for idx in range(20,len(aggregated_data)):
            current_row = aggregated_data[idx,:4]
            current_y = aggregated_data[idx,2]
            conditioning_data = aggregated_data[:idx,:4]

            torch_arr = torch.vstack((current_row, conditioning_data))
            y_and_neighbors = torch_arr[:, 2]
            locs = torch_arr[:, :2]

            cov_matrix = covariance_function(params=params, y= torch_arr, x= torch_arr)
            # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
            cov_xx = cov_matrix[1:, 1:]
            # cov_xx_inv = torch.linalg.inv(cov_xx)

            cov_yx = cov_matrix[0, 1:]

                    # Compute the log determinant of the covariance matrix
            # sign, log_det = torch.slogdet(cov_matrix)
            # if sign <= 0:
            #     raise ValueError("Covariance matrix is not positive definite")
        
            # Compute beta


            tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
            tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_and_neighbors))
            

            beta = torch.linalg.solve(tmp1, tmp2)

            mu = torch.matmul(locs, beta)
            mu_current = mu[0]
            mu_neighbors = mu[1:]

            # Mean and variance of y|x
            sigma = cov_matrix[0, 0]
            
            # cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
            cov_ygivenx = sigma - torch.matmul(cov_yx, torch.linalg.solve(cov_xx, cov_yx))
            # cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
            cond_mean = mu_current + torch.matmul(cov_yx, torch.linalg.solve( cov_xx,(y_and_neighbors[1:] - mu_neighbors) ) )
            
            alpha = current_y - cond_mean
            quad_form = alpha**2 * (1 / cov_ygivenx)
            log_det = torch.log(cov_ygivenx)
     
            neg_log_lik += 0.5 * (log_det + quad_form) 
        return neg_log_lik


    def vecchia_like_local_computer(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        
        for time_idx in range(len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            #cur_heads = current_np[:21, :]
            #neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(0, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors)
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
                    L_inv = self.cov_map[index]['L_inv']
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                    beta = torch.linalg.solve(tmp_for_beta, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                L = torch.linalg.cholesky(cov_matrix)
                L11 = L[:1, :1]
                L12 = torch.zeros(L[:1, 1:].shape)
                L21 = L[1:, :1]
                L22 = L[1:, 1:]
                L11_inv = torch.linalg.inv(L11)
                L22_inv = torch.linalg.inv(L22)

                # First block: [L11_inv, L12]
                upper_block = torch.cat((L11_inv, L12), dim=1)  # Concatenate along columns (dim=1)

                # Second block: [-torch.matmul(torch.matmul(L22_inv, L21), L11_inv), L22_inv]
                lower_left = -torch.matmul(torch.matmul(L22_inv, L21), L11_inv)
                lower_block = torch.cat((lower_left, L22_inv), dim=1)  # Concatenate along columns (dim=1)

                # Combine the upper and lower blocks
                L_inv = torch.cat((upper_block, lower_block), dim=0)  # Concatenate along rows (dim=0)

                cov_yx = cov_matrix[0, 1:]

                tmp1 = torch.matmul(L_inv, locs)
                tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                tmp_for_beta = torch.matmul(tmp1.T, tmp1)
                beta = torch.linalg.solve(tmp_for_beta, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)

             
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp_for_beta,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
                        'L_inv': L_inv,
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }

        return neg_log_lik  

## add base_list three times  when time_idx =0 1 >0

    def vecchia_extrapolate(self, params: torch.Tensor, covariance_function, cut_line=200) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0

        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            # cur_heads = current_np[:5, :]
            # neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
            
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index] , :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    y_arr = y_and_neighbors
                    tmp1 = tmp_for_beta
                    tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                    beta = torch.linalg.solve(tmp1, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp1,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
               
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }
        return neg_log_lik


    def vecchia_local_extra_base(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        
        for time_idx in range(len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            # cur_heads = current_np[:21, :]
            # neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(0, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) + base_list
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
            
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index] + base_list, :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    y_arr = y_and_neighbors
                    tmp1 = tmp_for_beta
                    tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                    beta = torch.linalg.solve(tmp1, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index]+ base_list, :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp1,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
               
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }
        return neg_log_lik
    

    def vecchia_interpolation_1to6(self, params: torch.Tensor, covariance_function, cut_line=200) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0 and time_idx<7:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    last_hour_np = self.input_map[self.key_list[time_idx +1]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)
                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik

In [4]:
class torch_vecchia_exp(matern_advec_beta_torch_vecchia):
    def __init__(self, analaysis_data_map: torch.Tensor, params: torch.Tensor, nns_map=nns_map, mm_cond_number=mm_cond_number):
        super().__init__(analaysis_data_map, params, nns_map, mm_cond_number)
        # Any additional initialization for dignosis class can go here


    def vecchia_interpolation_1to6(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0 and time_idx<7:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    last_hour_np = self.input_map[self.key_list[time_idx +1]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)
                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik

    def vecchia_b1(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik


    def vecchia_b2(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 1:
                    last_hour_np = self.input_map[self.key_list[time_idx -2]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)
                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik


    def vecchia_b3(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 1:
                    last_hour_np = self.input_map[self.key_list[time_idx -2]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 2:
                    last_hour_np = self.input_map[self.key_list[time_idx -3]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)
                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik
    

    def vecchia_b4(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 1:
                    last_hour_np = self.input_map[self.key_list[time_idx -2]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 2:
                    last_hour_np = self.input_map[self.key_list[time_idx -3]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 3:
                    last_hour_np = self.input_map[self.key_list[time_idx -4]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik

    def vecchia_b5(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 1:
                    last_hour_np = self.input_map[self.key_list[time_idx -2]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 2:
                    last_hour_np = self.input_map[self.key_list[time_idx -3]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 3:
                    last_hour_np = self.input_map[self.key_list[time_idx -4]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 4:
                    last_hour_np = self.input_map[self.key_list[time_idx -5]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik

    def vecchia_b6(self, params: torch.Tensor, covariance_function, cut_line) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        key_list = sorted(analysis_data_map)
        cut_line = cut_line
        heads = analysis_data_map[key_list[0]][:cut_line,:]
        for time_idx in range(1, len(analysis_data_map)):
            tmp = analysis_data_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        neg_log_lik += self.full_likelihood(params, heads, heads[:, 2], covariance_function)          
        
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 1:
                    last_hour_np = self.input_map[self.key_list[time_idx -2]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 2:
                    last_hour_np = self.input_map[self.key_list[time_idx -3]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 3:
                    last_hour_np = self.input_map[self.key_list[time_idx -4]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 4:
                    last_hour_np = self.input_map[self.key_list[time_idx -5]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if time_idx > 5:
                    last_hour_np = self.input_map[self.key_list[time_idx -6]]
                    # if index==200:
                    #     print(self.input_map[self.key_list[time_idx-6]])
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik


In [17]:
lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]

b = [0]*7

b1=b2=b3=b4=b5=b6=0
for day in range(1,5):
    print(f'\n day {day} \n')

    mm_cond_number = 10+day

    idx_for_datamap= [ 8*(day-1),8*day]

    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)
    instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

    out = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_ani)
    print(out)  # 15105


    out = instance.vecchia_local_full_cond(params,  instance.matern_cov_ani)
    print(out)  # 15105

    out = instance.vecchia_like_local_computer(params,  instance.matern_cov_ani)
    print(out)  # 15105

    out = instance.vecchia_local_extra_base(params,  instance.matern_cov_ani)
    print(out)  # 15105





 day 1 

tensor(678.9859, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(678.3950, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(672.0339, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(632.3896, dtype=torch.float64, grad_fn=<AddBackward0>)

 day 2 

tensor(628.3263, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(627.6324, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(624.5680, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(583.1291, dtype=torch.float64, grad_fn=<AddBackward0>)

 day 3 

tensor(721.9850, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(721.1641, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(706.3730, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(658.8450, dtype=torch.float64, grad_fn=<AddBackward0>)

 day 4 

tensor(676.1042, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(675.5429, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(676.1879, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(637.1947, dtype=torch.f

In [8]:
lat_lon_resolution = [10,10]

head100map = defaultdict(list)

headn = 10
b = [0]*7
b1=b2=b3=b4=b5=b6=0
for day in range(1,2):
    mm_cond_number = 10

    years = ['2024']
    month_range =[7,8]
    idx_for_datamap= [ 8*(day-1),8*day]

    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)
    instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)
    out = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_ani)
    print(out)  # 15105

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 5)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, headn)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 15)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 20)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 25)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 30)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 35)
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2
    print(f'day {day} finished')

tensor(2588.8355, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2539.4274, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2538.5730, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2538.1789, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2539.9501, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2542.8933, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2543.5865, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2544.8655, dtype=torch.float64, grad_fn=<AddBackward0>)
day 1 finished


In [14]:
lat_lon_resolution = [10,10]

head100map = defaultdict(list)
headn = 50

b = [0]*7
b1=b2=b3=b4=b5=b6=0
for day in range(1,32):
    mm_cond_number = 10

    years = ['2024']
    month_range =[7,8]
    idx_for_datamap= [ 8*(day-1),8*day]

    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)
    instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)
    out = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_ani)
    print(out)  # 15105


    # out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, headn)
    # print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2
    out1 = float('inf')

    out2 = instance.vecchia_b1(params, instance.matern_cov_ani, headn)
    print(out2) 

    out3 = instance.vecchia_b2(params, instance.matern_cov_ani, headn)
    print(out3) 

    out4 = instance.vecchia_b3(params, instance.matern_cov_ani, headn)
    print(out4) 

    out5 = instance.vecchia_b4(params, instance.matern_cov_ani, headn)
    print(out5) 

    out6 = instance.vecchia_b5(params, instance.matern_cov_ani, headn)
    print(out6) 

    out7 = instance.vecchia_b6(params, instance.matern_cov_ani, headn)
    print(out7) 


    tmp_result = [ torch.abs(out-out1), torch.abs(out-out2),torch.abs(out-out3),torch.abs(out-out4),torch.abs(out-out5),torch.abs(out-out6) ,torch.abs(out-out7) ]
    for i in range(6):
        if tmp_result[i] == min(tmp_result):
            b[i] +=1   #  mm_cond 10 , head10 then 3 2 11 4 1 4 0 

            print(f' day {day+1} full {out} best {i}-{min(tmp_result)}')



tensor(2588.8355, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2598.6200, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2596.4554, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.2973, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.4288, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.6245, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.7251, dtype=torch.float64, grad_fn=<AddBackward0>)
 day 2 full 2588.835494538167 best 2-7.619876935808406
tensor(2325.3057, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2338.3503, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.2224, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.3480, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.8504, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.7865, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.9979, dtype=torch.float64, grad_fn=<AddBackward0>)
 day 3 full 2325.3057231697794 best 2-11.916687282841394
tensor

In [15]:
b

[0, 5, 7, 6, 2, 5, 0]

In [ ]:
bmap = {}  #( cond number)
bmap[(10,10)] = [3,2,11,4,1,4,0]
bmap[(10,20)] = [9,4,8,3,1,2,0]
bmap[(10,30)] = [10,3,4,5,1,3,0]
bmap[(10,40)] = [12,4,4,4,2,3,0]
bmap[(10,50)] = [13,3,4,4,2,2,0]  # [0, 5, 7, 6, 2, 5, 0]

{(10, 10): [3, 2, 11, 4, 1, 4, 0],
 (10, 20): [9, 4, 8, 3, 1, 2, 0],
 (10, 30): [10, 3, 4, 5, 1, 3, 0]}

### Compare full likleihood vs vecchia interpolation vs vecchia extrapolation

In [38]:
lat_lon_resolution = [4,4]

# Create an empty DataFrame
head200df = pd.DataFrame(columns=['full', 'vecc_inter', 'vecc_extra', 'full-vecc_inter','full-vecc_extra'])

for day in range(1,32):
    mm_cond_number = 10

    years = ['2024']
    month_range =[7,8]
    idx_for_datamap= [ 8*(day-1),8*day]

    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)


    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)
    instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)
    out0 = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_ani).detach().numpy() 
    print(out0)  # 15105
    # out0 = instance.vecchia_local2(params, instance.matern_cov_ani)
    # print(out0)   # 15181.4775  15183 0,1 lage 15185 0,2

    out1 = instance.vecchia_interpolation_1to6(params, instance.matern_cov_ani, 200).detach().numpy() 
    print(out1)   # 15181.4775  15183 0,1 lage 15185 0,2
  
    out2 = instance.vecchia_like_local_computer(params, instance.matern_cov_ani).detach().numpy() 
    print(out2)
    head200df.loc[day-1] = [out0, out1, out2, out0-out1, out0-out2]
    print(f'day{day} finish')

14257.319
14233.447
14332.837
day1 finish
12877.236
12867.427
12949.4
day2 finish
15105.671
15112.495
15198.854
day3 finish
15057.407
15024.938
15142.105
day4 finish
12729.295
12719.305
12799.255
day5 finish
14974.811
14911.461
15026.632
day6 finish
15914.439
15905.117
15964.356
day7 finish
15345.43
15299.767
15427.701
day8 finish
13209.886
13204.401
13292.492
day9 finish
15197.731
15179.452
15288.105
day10 finish
13394.018
13358.698
13453.695
day11 finish
12367.97
12354.931
12420.804
day12 finish
12334.455
12334.198
12398.205
day13 finish
13388.797
13357.456
13426.342
day14 finish
11979.054
11971.41
12037.568
day15 finish
12619.898
12600.382
12677.345
day16 finish
14524.198
14494.066
14582.043
day17 finish
13897.533
13883.741
13965.368
day18 finish
13206.887
13188.489
13267.975
day19 finish
13289.11
13276.595
13340.859
day20 finish
13417.07
13425.823
13504.782
day21 finish
13321.848
13321.367
13379.887
day22 finish
11662.559
11667.87
11710.672
day23 finish
12236.074
12234.279
12310.16

In [ ]:
input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates"

# Generate date range
date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(head200df):
    head200df.index = date_range

# Save DataFrame to CSV
output_filename = 'head200_full_vs_vecc_1250_july24.csv'
output_csv_path = os.path.join(input_path, output_filename)
head200df.to_csv(output_csv_path, index=False)

Fit the model

In [95]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_local4(params, instance.matern_cov_ani, 200 )
    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(600):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
        break
    
    prev_loss = loss.item()

print('Training complete.') 

Epoch 1, Gradients: [  -9.955391  207.88681    -2.595461 -169.72203   273.39944   771.1119
  -79.33867 ]
 Loss: 15135.7587890625, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [ -1.4867018    0.10161591   1.1231489    3.3106918  -11.375813
  -7.5312614    7.9387875 ]
 Loss: 15014.2587890625, Parameters: [ 2.5338207e+01  1.3597493e+00  1.7280715e+00  1.9441580e-02
 -1.6756412e-01  1.8955134e-01  3.3110123e+00]
Converged at epoch 189
Epoch 190, Gradients: [-1.2997863  -0.27633095 -0.09813547  0.49712753 -0.5989456  -1.545433
 -0.09783697]
 Loss: 15012.7626953125, Parameters: [ 2.5795250e+01  1.3089702e+00  1.6588148e+00  1.8806925e-02
 -1.6661680e-01  1.9616681e-01  3.1115689e+00]
Training complete.


In [ ]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_local3(params, instance.matern_cov_ani )
    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(500):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
        break
    
    prev_loss = loss.item()

print('Training complete.') 

 
# vecchia local 2  332   epo 33.4s   25.55 2.61 2.68 0.16 0.03 2.7
#    24.89 2.06 2.24 1.36e-2 -5.63 e-2 0.10113 3.75
# vecchia cholesky local 380 epo 43.2  25 2.61 2.68 0.16 0.03 2.7

Epoch 1, Gradients: [-10.6224985 200.28781    -6.4957714 -78.33738   149.36243   605.0917
 -81.693665 ]
 Loss: 15183.4765625, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-1.9947629  4.9151707  6.4719105 -1.286026  -2.0892706  2.9007835
  1.0226293]
 Loss: 15077.75, Parameters: [25.32368     1.3661038   1.7502507   0.04647188 -0.12653434  0.17788467
  3.327231  ]
Converged at epoch 135
Training complete.


Experiment results

## 200 x 8

lr 0.001 without scheduler  same as lr, step_size, gamma  0.01 40 0.5  (9.8s)

 Loss: 2549.066650390625, full Parameters: [ 2.48777485e+01  2.05998826e+00  2.16013098e+00  2.20775465e-03
 -7.89414570e-02  1.05411254e-01  3.75236106e+00]

 lr 0.01  step size 40  betas 0.9 , 0.8 gamma 0.9  30 s

  Loss: 2547.1728515625, full Parameters: [ 2.7377291e+01  2.2077193e+00  2.3204505e+00  1.0307773e-03
 -8.0311157e-02  9.8579854e-02  3.6677265e+00]

 lr 0.01  step size 10 betas 0.9 , 0.8 gamma 0.9  30 s
  Loss: 2548.87841796875, full Parameters: [ 2.5092268e+01  2.0689390e+00  2.1694989e+00  2.0285936e-03
 -7.9028614e-02  1.0501490e-01  3.7373385e+00]
Training full likelihood complete.   11.8 sc

 lr 0.01  step size 20 betas 0.9 , 0.8 gamma 0.9  30 s
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]



lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9
 Loss: 2548.18603515625, full Parameters: [ 2.5938652e+01  2.1110108e+00  2.2155209e+00  1.5893303e-03
 -7.9482891e-02  1.0297947e-01  3.6958976e+00]
 21.6

lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]
 22.9 s

lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9
Loss: 2548.95361328125, full Parameters: [ 2.5118145e+01  1.9827319e+00  2.0768294e+00  1.0898338e-03
 -8.0070712e-02  1.1034889e-01  3.5647078e+00]


## 1250 x 8

1250* 8 55m using constant learning rate 0.0001 
Loss: 14068.798828125, full Parameters: [ 2.46198387e+01  1.61719894e+00  1.76454413e+00  8.55297223e-03
 -1.08275235e-01  1.28809512e-01  2.80795789e+00]

1250* 8 10m 32s
lr 0.01  step size 40 beta 0.9 0.8 gamma 0.9
  Loss: 14068.1953125, full Parameters: [ 2.5030930e+01  1.6107724e+00  1.7573007e+00  8.8407323e-03
 -1.0820019e-01  1.2936097e-01  2.7430327e+00]
Training full likelihood complete.

9m 33s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 14068.29296875, full Parameters: 
 [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03 -1.0737537e-01  1.2953614e-01 
  2.7420275e+00]
Training full likelihood complete.

#### high resolution data might benefits from larger step size high resolution data often provides 
#### more stable gradients, so larger step size less likely to cause significant fluctuations
14n 41.8s
lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9

FINAL STATE: Epoch 199, 
 Loss: 14068.8828125, full Parameters: 
 [ 2.4707581e+01  1.6489888e+00  1.7993137e+00  8.4043797e-03 -1.0836436e-01  1.2655504e-01  
 2.8416286e+00]

#### beta 0.9 0.99 might be too conservative for high resolution data
13m 44.8s
lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9

 Loss: 14068.318359375, full Parameters: [ 2.4938175e+01  1.6203119e+00  1.7678342e+00  8.6686825e-03
 -1.0813228e-01  1.2845081e-01  2.7731323e+00]


18m
lr 0.01  step size 40 beta 0.9 0.99 gamma 0.9

 Loss: 14067.970703125, full Parameters: [ 2.5205673e+01  1.6159834e+00  1.7630767e+00  8.7957922e-03
 -1.0802399e-01  1.2862283e-01  2.7390635e+00]

9m 52s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9

Loss: 14068.29296875, full Parameters: [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03
 -1.0737537e-01  1.2953614e-01  2.7420275e+00]
Training full likelihood complete.